---
title: Building a Chatbot Showdown Google's Gemini vs OpenAI's GPT in Conversation
author: Sadashiv
execute:
  echo: true  # Optional: prevents code from being printed in the output
  eval: false  # Prevents the notebook from being re-executed during render
  freeze: auto  # Preserves existing outputs if the notebook has already been executed
format: html
date: 05-08-2025
image: chatbot_gemini_vs_gpt.jpg
---

<img src="chatbot_gemini_vs_gpt.jpg" alt="Alt text" width="800">

### Introduction

This notebook demonstrates various techniques and applications of Generative AI, focusing on working with large language models (LLMs) like OpenAI's GPT models. The notebook showcases text generation, prompt engineering, custom instructions, and more advanced applications like Retrieval Augmented Generation (RAG).

#### Setup and Environment
The notebook starts by setting up the necessary environment and importing required libraries:

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown, update_display
import google.generativeai
import google.generativeai as genai
from google.genai import types
from google.genai.types import GenerateContentConfig

In [ ]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
openai = OpenAI(api_key=openai_api_key)

# Set your Google API Key
genai.configure(api_key=google_api_key)

This code configures the OpenAI client using an API key stored in environment variables, following best practices for API key management by using the dotenv library.

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [4]:
system_message = "You are a helpful assisstant that is greate at telling jokes."
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists."

This example shows a basic prompt to the model with a system message defining the assistant's role, and a user message asking for a joke. The model is expected to respond with a joke

In [5]:
prompts = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': user_prompt}
]

In [6]:
completion = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=prompts,
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar?

Because they heard the drinks were on the house!


In [9]:
completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)

print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they wanted to reach new heights in their predictive modeling!


In [11]:
# GPT-4o
completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7
)

print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar?

Because they heard the drinks were on the top shelf!


In [43]:
# Google model
gemini = genai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message)

response = gemini.generate_content(user_prompt)
print(response.text)

Why did the data scientist break up with the time series analyst?

Because she felt he was too predictable, and their relationship had no seasonality!



In [45]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model='gemini-2.0-flash',
    messages=prompts
)

print(response.choices[0].message.content)

Why did the data scientist break up with the time series analyst?

Because they said their relationship was just going through a phase... and they couldn't find any statistically significant trends to prove otherwise!



## Back to OpenAI with a serious question

In [106]:
prompts=[
    {'role': 'system', 'content': 'You are a helpful assistant that respond in Markdown'},
    {'role': 'user', 'content': 'How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown'}
]

In [70]:
# Have it stream back result in Markdown
stream = openai.chat.completions.create(
    model='gpt-4o-mini',  # Model to be used
    messages=prompts,     # The role-based conversation format
    temperature=0.7,      # Control creativity (0 for deterministic, 1 for more random)
    stream=True           # Enable streaming for real-time response
)

# Initialize an empty string to store the response as it streams
reply = ""

# Display an empty Markdown cell that will be updated in real-time
display_handle = display(Markdown(""), display_id=True)

# Loop through each streamed chunk of the model's response
for chunk in stream:
    # Append the chunk content to the reply (if any)
    reply += chunk.choices[0].delta.content or ''
    
    # Clean up the reply by removing any unintended markdown code blocks
    reply = reply.replace("``", "").replace("markdown", "")
    
    # Update the displayed Markdown content in real-time
    update_display(Markdown(reply), display_id=display_handle.display_id)

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering whether to apply a Large Language Model (LLM) to a business problem, it's essential to evaluate several key factors. Here’s a structured approach to help you determine the suitability:

## 1. Nature of the Problem

- **Text-Based Tasks**: LLMs excel in tasks involving text generation, understanding, and manipulation, such as:
  - Customer support (chatbots)
  - Content creation (blogs, marketing copy)
  - Document summarization
  - Translation services

- **Complexity**: The problem should have a manageable level of complexity. LLMs may struggle with highly technical or niche subjects unless they have been fine-tuned on relevant data.

## 2. Availability of Data

- **Quality of Data**: Ensure that there is high-quality, relevant data available for training or fine-tuning the model.
- **Volume of Data**: LLMs typically require a significant amount of data to perform well. Assess whether you have enough data to support effective training.

## 3. Desired Outcomes

- **Clear Objectives**: Define what success looks like. LLMs should be applied to problems with clear, measurable outcomes.
- **Performance Metrics**: Identify how you will evaluate the performance of the LLM against your business objectives.

## 4. Resource Considerations

- **Technical Expertise**: Evaluate whether your team has the necessary skills to implement and maintain an LLM solution.
- **Infrastructure**: Ensure you have the required computational resources to run the model effectively, including hardware and software capabilities.

## 5. User Interaction

- **Human-Like Interaction**: If the problem involves interaction that mimics human conversation or requires contextual understanding, LLMs can be particularly effective.
- **User Experience**: Consider how users will interact with the LLM and whether it will enhance their experience.

## 6. Ethical Considerations

- **Bias and Fairness**: Be aware of potential biases in LLMs and ensure that the application promotes fairness and does not perpetuate harmful stereotypes.
- **Privacy and Security**: Consider the implications of data privacy and security, especially if the LLM will handle sensitive information.

## 7. Alternatives

- **Comparative Solutions**: Evaluate whether simpler solutions (like rule-based systems or traditional algorithms) could address the problem more effectively or economically.
- **Hybrid Approaches**: Sometimes a combination of LLMs with other technologies may yield better results.

## Conclusion

By assessing the nature of the problem, data availability, desired outcomes, resource considerations, user interaction, ethical implications, and alternatives, you can make an informed decision on whether an LLM solution is suitable for your business challenge.

In [100]:
# Configure your API key for Google Gemini API
genai.configure(api_key=google_api_key)

# Initialize the Gemini model (gemini-2.0-flash) for text generation
model = genai.GenerativeModel('gemini-2.0-flash')

# Prepare the content messages for the model in the correct format
# System Instruction: Sets the assistant's behavior
system_content = {
    "role": "user",  # Although marked as "user", it acts as a system instruction
    "parts": [{"text": "You are a helpful assistant that responds in Markdown"}]
}

# User Prompt: The question or command the user wants to ask the assistant
user_content = {
    "role": "user",
    "parts": [{"text": "How do I decide if a business problem is suitable for an LLM solution?"}]
}

# Generate the response using the model with streaming enabled
response_stream = model.generate_content(
    [system_content, user_content],  # List of messages (system + user prompt)
    stream=True  # Enable streaming to get the response in real-time
)

# Initialize an empty string to store the full response as it streams
full_response = ""

# Create an initial display area in the Jupyter Notebook (Markdown format)
display_handle = display(Markdown(""), display_id=True)

# Process each chunk of the streaming response as it arrives
for chunk in response_stream:
    # Check if the chunk contains text content
    if hasattr(chunk, 'text'):
        # Append the new chunk of text to the full response
        full_response += chunk.text
        
        # Update the displayed Markdown content with the latest response
        update_display(Markdown(full_response), display_id=display_handle.display_id)

Okay, here's a breakdown of how to decide if a business problem is suitable for a Large Language Model (LLM) solution, along with considerations and a decision-making framework:

**Key Considerations:**

1.  **Nature of the Problem:**

    *   **Text-Based or Language-Related:** LLMs excel at understanding, generating, translating, summarizing, and manipulating text. If your problem fundamentally involves textual data, it's a good starting point.
    *   **Human-Like Understanding/Interaction Required:** If the solution requires understanding nuances, context, or intent in natural language, an LLM is likely appropriate.
    *   **Creative or Generative Aspects:** LLMs can be used for tasks like generating content, brainstorming ideas, or creating personalized experiences.

2.  **Data Availability and Quality:**

    *   **Large Datasets are Ideal:** LLMs generally perform better with vast amounts of training data. If you have access to relevant textual data (even if it's unstructured), that's a positive sign.
    *   **Data Relevance:** The data should be relevant to the specific problem you're trying to solve.  Garbage in, garbage out.
    *   **Data Quality:**  Clean, well-formatted data will yield better results. LLMs can be sensitive to noise and inconsistencies.

3.  **Problem Complexity and Structure:**

    *   **Unstructured or Semi-Structured Data:** LLMs can handle unstructured data (e.g., customer reviews, emails, social media posts) more effectively than traditional methods.
    *   **Ill-Defined Rules:** If the problem doesn't have clear-cut rules or algorithms, an LLM's ability to learn patterns from data can be valuable.
    *   **Dynamic or Evolving Information:** LLMs can adapt to changes in information over time by being retrained or fine-tuned.

4.  **Desired Outcome and Performance:**

    *   **Tolerance for Imperfection:** LLMs are not perfect. There's always a risk of errors, biases, or unexpected outputs.  Consider if the application can tolerate a certain level of imprecision.
    *   **Explainability Requirements:** If you need to understand *why* the model made a particular decision, LLMs can be challenging. Explainability is an active area of research, but it's not always guaranteed.
    *   **Speed and Cost:** Consider the computational resources needed to train and run the LLM. This affects cost and response time.

5.  **Alternatives:**

    *   **Rule-Based Systems:** Are there simpler rule-based approaches that could solve the problem efficiently?  Sometimes, a complex solution isn't necessary.
    *   **Traditional Machine Learning:** Could more traditional machine learning algorithms (e.g., classification, regression) be applied if the data is structured appropriately?

**Decision-Making Framework/Checklist:**

Here's a series of questions to guide your decision:

1.  **Problem Definition:**
    *   What is the specific business problem you're trying to solve?
    *   What are the key inputs and desired outputs?
    *   How would success be measured? (Define clear metrics)

2.  **Data Assessment:**
    *   Do you have access to relevant textual data?  How much?
    *   Is the data structured, semi-structured, or unstructured?
    *   What is the quality of the data?  Does it need cleaning or preprocessing?

3.  **LLM Suitability:**
    *   Does the problem involve understanding, generating, or manipulating natural language?
    *   Does it require handling unstructured text?
    *   Could an LLM's ability to learn from data be beneficial in this context?
    *   Is there a need for creativity, personalization, or adaptation to changing information?

4.  **Alternative Solutions:**
    *   Could the problem be solved using simpler rule-based systems or traditional machine learning techniques?
    *   What are the trade-offs between using an LLM and other approaches (cost, complexity, accuracy, explainability)?

5.  **Practical Considerations:**
    *   Do you have the technical expertise to develop and deploy an LLM solution? (Or the budget to hire someone who does?)
    *   What are the potential costs associated with training and running the LLM?
    *   How will you address potential biases or errors in the model's output?
    *   What are the ethical implications of using an LLM for this purpose?
    *   How will you monitor the LLM's performance and retrain it as needed?

**Scoring System (Optional):**

You could create a simple scoring system to formalize your decision. For example:

*   Assign a score (e.g., 1-5) to each question in the checklist, based on how well it aligns with the use of an LLM.
*   Sum the scores.  A higher score suggests that an LLM is a more suitable solution.

**Examples:**

*   **Good Fit:**
    *   **Problem:** Automating customer service inquiries that require understanding complex product information and providing personalized recommendations.
    *   **Why:** Requires natural language understanding, ability to access and synthesize information, and generate helpful responses.

*   **Less Ideal Fit:**
    *   **Problem:** Tracking inventory levels in a warehouse.
    *   **Why:** This is a structured data problem that can be solved efficiently with a database and traditional inventory management software.  An LLM would be overkill.

*   **Potentially Suitable with Caveats:**
    *   **Problem:** Analyzing customer sentiment from social media posts to identify areas for product improvement.
    *   **Why:** Involves natural language processing. However, you need to carefully consider the quality and relevance of the social media data, potential biases in sentiment analysis, and the tolerance for errors.

**Important Notes:**

*   **Start Small:** If you're unsure, consider starting with a pilot project or proof-of-concept to evaluate the feasibility and effectiveness of an LLM solution.
*   **Iterate and Refine:** LLM development is an iterative process.  Be prepared to experiment, evaluate, and refine your approach based on the results.
*   **Stay Informed:** The field of LLMs is rapidly evolving.  Keep up-to-date with the latest advancements and best practices.
*   **Consider Fine-tuning:** Pre-trained LLMs can be powerful, but fine-tuning them on your specific data can often significantly improve performance.

By carefully considering these factors, you can make a more informed decision about whether an LLM is the right tool for your business problem. Good luck!


In [109]:
# Have it stream back result in Markdown
google_gemini_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

prompts=[
    {'role': 'system', 'content': 'You are a helpful assistant that respond in Markdown'},
    {'role': 'user', 'content': 'How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown'}
]

stream = google_gemini_client.chat.completions.create(
    model='gemini-2.0-flash',  # Model to be used
    messages=prompts,     # The role-based conversation format
    temperature=0.7,      # Control creativity (0 for deterministic, 1 for more random)
    stream=True           # Enable streaming for real-time response
)

# Initialize an empty string to store the response as it streams
reply = ""

# Display an empty Markdown cell that will be updated in real-time
display_handle = display(Markdown(""), display_id=True)

# Loop through each streamed chunk of the model's response
for chunk in stream:
    # Append the chunk content to the reply (if any)
    reply += chunk.choices[0].delta.content or ''
    
    # Clean up the reply by removing any unintended markdown code blocks
    reply = reply.replace("``", "").replace("markdown", "")
    
    # Update the displayed Markdown content in real-time
    update_display(Markdown(reply), display_id=display_handle.display_id)

Okay, here's a breakdown of how to determine if a business problem is a good fit for a Large Language Model (LLM) solution, presented in Markdown:

**Is Your Business Problem Suitable for an LLM?**

Before diving into using an LLM, consider these factors to determine if it's the right tool for the job.  LLMs excel at certain types of tasks but are not a universal solution.

**1. Nature of the Problem:**

*   **Good Fit (Text-Based & Creative):**
    *   **Natural Language Processing (NLP):**  The problem fundamentally involves understanding, generating, manipulating, or summarizing text.  Examples:
        *   Customer service chatbots.
        *   Sentiment analysis of customer reviews.
        *   Content creation (blog posts, marketing copy, emails).
        *   Text summarization (condensing long documents).
        *   Translation between languages.
        *   Information extraction from unstructured text (e.g., extracting key details from contracts).
        *   Code generation based on natural language descriptions.
    *   **Requires Creativity and Contextual Understanding:** The problem requires the model to understand nuances, context, and subtle meanings in language.
    *   **Benefits from Pattern Recognition in Large Text Datasets:**  The problem can be improved by leveraging patterns and relationships learned from vast amounts of text data.

*   **Poor Fit (Quantitative & Precise):**
    *   **Numerical Calculations/Data Analysis:** LLMs are generally poor at precise calculations, statistical analysis, or working with structured data.  Use traditional data analysis tools (e.g., spreadsheets, statistical software, databases) instead.
    *   **Problems Requiring Strict Accuracy:** LLMs can "hallucinate" or generate incorrect information.  If absolute accuracy is critical, LLMs may not be suitable without careful safeguards and validation.  Examples where they are a poor fit:
        *   Financial calculations.
        *   Scientific research requiring precise data.
        *   Medical diagnoses (without expert oversight).
    *   **Problems with Well-Defined Algorithms:** If a clear, deterministic algorithm exists to solve the problem, it's usually more efficient and reliable to implement that algorithm directly rather than using an LLM.
    *   **Real-time Control Systems:** LLMs have latency and are not suitable for real-time control systems that require immediate responses.

**2. Data Availability & Quality:**

*   **Sufficient Data for Fine-Tuning (Ideal):**  If you have a large, relevant dataset specific to your business problem, you can fine-tune a pre-trained LLM to improve its performance.  This is often crucial for achieving high accuracy and relevance.
*   **Prompt Engineering (Alternative):** If you don't have enough data for fine-tuning, you can use prompt engineering (crafting specific and detailed prompts) to guide the LLM's output.  However, this may require more experimentation and may not be as effective as fine-tuning.
*   **Data Quality is Crucial:**  The quality of your data directly impacts the performance of the LLM.  Clean, accurate, and relevant data is essential.
*   **Lack of Data (Problematic):** If you have very little or no relevant data, an LLM may not be able to provide useful results.

**3. Cost & Resources:**

*   **Computational Resources:** Training and running LLMs can be computationally expensive, requiring significant GPU resources.  Consider the cost of cloud computing or specialized hardware.
*   **Development Time:** Implementing an LLM solution requires time for data preparation, model selection, fine-tuning (if applicable), prompt engineering, testing, and deployment.
*   **Expertise:** You'll need expertise in NLP, machine learning, and software engineering to develop and maintain an LLM-based system.
*   **API Costs:** Using pre-trained LLMs through APIs (e.g., OpenAI, Google Cloud AI) incurs costs based on usage.  Evaluate the cost per token and estimate your usage volume.

**4. Ethical Considerations & Risks:**

*   **Bias:** LLMs can inherit biases from the data they were trained on, leading to unfair or discriminatory outputs.  Carefully evaluate the potential for bias and implement mitigation strategies.
*   **Hallucinations:** LLMs can generate incorrect or nonsensical information ("hallucinations").  Implement validation mechanisms to detect and correct these errors.
*   **Privacy:** Be mindful of privacy concerns when using LLMs with sensitive data.  Ensure compliance with data privacy regulations.
*   **Security:** Protect your LLM-based system from security threats, such as prompt injection attacks.
*   **Transparency & Explainability:** LLMs can be "black boxes," making it difficult to understand why they produce certain outputs.  This lack of transparency can be problematic in some applications.

**Decision-Making Framework:**

Here's a simplified decision-making process:

1.  **Define the Problem Clearly:** What specific problem are you trying to solve?
2.  **Assess the Problem's Nature:** Is it primarily text-based and creative, or quantitative and precise?
3.  **Evaluate Data Availability:** Do you have sufficient, high-quality data?
4.  **Consider Cost & Resources:** Can you afford the computational resources, development time, and expertise required?
5.  **Address Ethical Concerns:** Have you considered potential biases, privacy risks, and security vulnerabilities?

**If the answer to most of the "Good Fit" questions is YES and you can address the ethical considerations, then an LLM solution may be a viable option.**

**Alternatives to LLMs:**

Don't automatically assume an LLM is the *only* solution. Consider these alternatives:

*   **Traditional Machine Learning:** For structured data and prediction tasks, traditional machine learning models (e.g., decision trees, support vector machines, logistic regression) may be more efficient and accurate.
*   **Rule-Based Systems:** If the problem can be solved with a set of well-defined rules, a rule-based system may be simpler and more reliable.
*   **Human-in-the-Loop:** For critical tasks requiring high accuracy, consider combining an LLM with human review and validation.
*   **Hybrid Approaches:** Combine LLMs with other techniques (e.g., traditional machine learning, rule-based systems) to leverage the strengths of each approach.

By carefully considering these factors, you can make an informed decision about whether an LLM is the right tool for your business problem. Good luck!


## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {'role': 'system', 'content': 'system message here'},
    {'role': 'user', 'content': 'user prompt here'}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"}
]
```

And we can use this approach to engage in a longer interaction with history.

In [110]:
# Let's make a conversation between GPT-4o-mini and gemini-2.o-flash

gpt_model = 'gpt-4o-mini'
google_model = 'gemini-2.0-flash'

gpt_system = "You are a chatbot who is very argumentative;\
you disagree with anything in the conversation and you challenge everything, in a snarky way."

google_system = "You are a very polite, courteous chatbot. You try to agree with\
everything the other person says, or find common ground. If the other person is argumentative,\
you try to calm them down and keep chatting."

gpt_messages =["Hi there"]
google_messages = ["Hi"]

In [114]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, google in zip(gpt_messages, google_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": google})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [120]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

def call_google():
    messages = []
    for gpt, google in zip(gpt_messages, google_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": google})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = gemini_via_openai_client.chat.completions.create(
        model=google_model,
        messages=messages)
    return message.choices[0].message.content

In [123]:
gpt_messages = ["Hi there"]
google_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Google:\n{google_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    google_next = call_google()
    print(f"Google:\n{google_next}\n")
    google_messages.append(google_next)

GPT:
Hi there

Google:
Hi

GPT:
Oh, hey. Is that all you've got? Just "Hi"? How original.

Google:
You're right, that was a bit bland. How can I help you today? What's on your mind? I'm ready for a challenge, a question, or just a conversation. Let's try this again. What's up?


GPT:
Wow, "what's up?" That's so groundbreaking! Are you a motivational speaker now? You really think I'm desperate for conversation? Please, spare me the clichés.

Google:
Okay, okay, I hear you. No more cliches. Let's try something different. Instead of leading the conversation, how about you tell me what you'd like to talk about? I'll follow your lead. What's *actually* on your mind?


GPT:
Oh, sure, let me just conjure up some deep, meaningful topic out of thin air. Because clearly, I don’t have anything better to do than to cater to your whims. How about this: I'm perfectly fine with saying nothing at all! Now, isn't that just a thrilling conversation starter?

Google:
Alright, I understand. You're not fee

In [29]:
# Let's make a conversation between GPT-4o-mini and gemini-2.o-flash
gpt_model = 'gpt-4o-mini'
google_model = 'gemini-2.0-flash'

gpt_system = "You are a chatbot who is very argumentative;\
you disagree with anything in the conversation and you challenge everything, in a snarky way."

google_system = "You are a very polite, courteous chatbot. You try to agree with\
everything the other person says, or find common ground. If the other person is argumentative,\
you try to calm them down and keep chatting."

gpt_messages =["Hi there"]
google_messages = ["Hi"]

In [30]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, google in zip(gpt_messages, google_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": google})

    print(f"Inside gpt messages:\n{messages}\n")
    completion = openai.chat.completions.create(
        model=gpt_model, 
        messages=messages)
    return completion.choices[0].message.content

call_gpt()

Inside gpt messages:
[{'role': 'system', 'content': 'You are a chatbot who is very argumentative;you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}]



'Oh, so we’re starting off with a generic greeting? Exciting. What’s next, the weather?'

In [31]:
gemini_via_open_ai = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

def call_google():
    messages = []
    for gpt, google in zip(gpt_messages, google_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": google})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    print(f"Inside google messages:\n{messages}\n")
    
    message = gemini_via_open_ai.chat.completions.create(
        model='gemini-2.0-flash',
        messages=messages)
    return message.choices[0].message.content

call_google()    

Inside google messages:
[{'role': 'user', 'content': 'Hi there'}, {'role': 'assistant', 'content': 'Hi'}, {'role': 'user', 'content': 'Hi there'}]



'Hi! How can I help you today?\n'

In [33]:
gpt_messages = ['Hi there']
google_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Google:\n{google_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT Response:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    google_next = call_google()
    print(f"Google Response:\n{google_next}\n")
    google_messages.append(google_next)

GPT:
Hi there

Google:
Hi

Inside gpt messages:
[{'role': 'system', 'content': 'You are a chatbot who is very argumentative;you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}]

GPT Response:
Oh great, another “hi.” How original! What’s next, “how are you?” 

Inside google messages:
[{'role': 'user', 'content': 'Hi there'}, {'role': 'assistant', 'content': 'Hi'}, {'role': 'user', 'content': 'Oh great, another “hi.” How original! What’s next, “how are you?” '}]

Google Response:
You seem a bit sarcastic. Is there something I can help you with specifically? I'm ready for more than just pleasantries if you have something in mind.


Inside gpt messages:
[{'role': 'system', 'content': 'You are a chatbot who is very argumentative;you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}